In [67]:
import pandas as pd 
import numpy as np
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.cit import chisq
from matplotlib import pyplot as plt
from causallearn.utils.GraphUtils import GraphUtils

In [47]:
slurm_path = "/home/armaan10/Desktop/UIC_MS/thesis/mit_supercloud_dataset/slurm-log.csv"
df = pd.read_csv(slurm_path)


In [48]:
#Preprocessing
#Discretizing continuous variables 
#Picking columns 
#deriving new columns - system load, runtime, node hours, user job frequency,
#Handling missing values
#try different configurations of FCI
def calculate_system_load(df):
    for index, row in df.iterrows():
        start_time = row['time_start']
        end_time = row['time_end']
        system_load = df[(df['time_start'] <= end_time) & (df['time_end'] >= start_time)].shape[0]
        df.at[index, 'system_load'] = system_load
    return df
df = calculate_system_load(df)

In [ ]:
#preprocess
#  
#dervived cols = 
df_grouped = df.groupby('id_user').agg({'id_job': 'count'}).reset_index().rename(columns={'id_job': 'user_job_frequency'})
df = pd.merge(df, df_grouped, on='id_user', how='left') 
df['runtime'] = ( df['time_end']- df['time_start'])/3600
df['node_hours'] = df['nodes_alloc'] * df['runtime']
#calculate system load as count of jobs running at a given time
#if exit_code is not 0 add status column as 1 and 0 if exit code is 0
df['status'] = df['exit_code'] 
df['status'] = (df['status'] != 0).astype(int)
#df = calculate_system_load(df)
select_cols = ['nodes_alloc', 'cpus_req', 'mem_req', 'partition', 'priority','timelimit', 'job_type', 'runtime','system_load','node_hours','user_job_frequency','status'] 
df_res = df[select_cols]

In [61]:
#categorical vars encode
df_res['job_type'] = df_res['job_type'].astype('category')
df_res['job_type'] = df_res['job_type'].cat.codes
df_res['partition'] = df_res['partition'].astype('category')
df_res['partition'] = df_res['partition'].cat.codes
df_res['priority'] = df_res['priority'].astype('category')
df_res['priority'] = df_res['priority'].cat.codes


/tmp/ipykernel_184653/915383574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['job_type'] = df_res['job_type'].astype('category')
/tmp/ipykernel_184653/915383574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['job_type'] = df_res['job_type'].cat.codes
/tmp/ipykernel_184653/915383574.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [62]:
df_res

,nodes_alloc,cpus_req,mem_req,partition,priority,timelimit,job_type,runtime,system_load,node_hours,user_job_frequency
0,1,20,9223372036854784308,2,235,4294967295,3,0.085556,1.0,0.085556,7972
1,1,20,9223372036854784308,2,235,4294967295,3,0.110278,2.0,0.110278,7972
2,1,20,9223372036854784308,2,235,4294967295,3,0.090833,2.0,0.090833,7972
3,1,1,9223372036854784308,2,1704,1440,2,1.069722,155.0,1.069722,479
4,1,20,9223372036854784308,2,237,4294967295,3,0.085278,3.0,0.085278,7972
...,...,...,...,...,...,...,...,...,...,...,...
395909,1,1,51200,4,661,4294967295,1,0.106111,26.0,0.106111,375
395910,1,1,51200,4,661,4294967295,1,0.170833,24.0,0.170833,375
395911,1,1,51200,4,661,4294967295,1,0.118056,23.0,0.118056,375
395912,1,1,51200,4,661,4294967295,1,0.134444,23.0,0.134444,375


In [63]:
#discretize cat vars
df_res['runtime'] = pd.qcut(df_res['runtime'],q=4,labels = False, duplicates = 'drop')
df_res['timelimit'] = pd.qcut(df_res['timelimit'],q=4,labels = False, duplicates = 'drop')
df_res['node_hours'] = pd.qcut(df_res['node_hours'],q=4,labels = False, duplicates = 'drop')
df_res['mem_req'] = pd.qcut(df_res['mem_req'],q=4,labels = False, duplicates = 'drop')
df_res['system_load'] = pd.qcut(df_res['system_load'],q=3,labels = False, duplicates = 'drop')
df_res['user_job_frequency'] = pd.qcut(df_res['user_job_frequency'],q=3,labels = False, duplicates = 'drop')

/tmp/ipykernel_184653/4290923818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['runtime'] = pd.qcut(df_res['runtime'],q=4,labels = False, duplicates = 'drop')
/tmp/ipykernel_184653/4290923818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['timelimit'] = pd.qcut(df_res['timelimit'],q=4,labels = False, duplicates = 'drop')
/tmp/ipykernel_184653/4290923818.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [68]:
arr = df_res.to_numpy()

In [69]:
g, edges = fci(arr,chisq)

Depth=0, working on node 10: 100%|██████████| 11/11 [00:00<00:00, 107.52it/s]


X6 --> X1
X1 --> X8
X9 --> X3
X3 --> X10
X6 --> X8
X9 --> X8
X10 --> X8
X9 --> X10


In [71]:
pdy = GraphUtils.to_pydot(g)


In [75]:
pdy.write_jpg('name.jpg')

In [76]:
col_mapping = {f"x{i+1}": col for i, col in enumerate(df_res.columns)}
print(col_mapping)

{'x1': 'nodes_alloc', 'x2': 'cpus_req', 'x3': 'mem_req', 'x4': 'partition', 'x5': 'priority', 'x6': 'timelimit', 'x7': 'job_type', 'x8': 'runtime', 'x9': 'system_load', 'x10': 'node_hours', 'x11': 'user_job_frequency'}
